In [3]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Define transforms for EfficientNetV2
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to 64x64
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])

# Load dataset
dataset = datasets.ImageFolder(root='/home/vamsi/cv/project/neww/Image-Super-Resolution-via-Iterative-Refinement/classs/share_it100', transform=transform)

# Split dataset into train and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)


In [6]:

import timm
import torch.nn as nn
import torch

# Load EfficientNetV2-Small pre-trained model
model = timm.create_model('efficientnetv2_s', pretrained=False)

# Modify the classifier head
num_classes = len(dataset.classes)  # Number of classes in the dataset
model.classifier = nn.Linear(model.classifier.in_features, num_classes)

# Move the model to the device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


EfficientNet(
  (conv_stem): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): ConvBnAct(
        (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
      (1): ConvBnAct(
        (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
    )
    (1): Seq

In [7]:
from torch.optim import Adam

criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = Adam(model.parameters(), lr=0.001)


In [8]:
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    print(f"Epoch {epoch + 1}/{epochs}")
    
    # Training phase
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_loss = running_loss / len(train_loader)
    print(f"Training Loss: {train_loss:.4f}")

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Accuracy calculation
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = 100 * correct / total

    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%\n")


Epoch 1/10
Training Loss: 2.3242
Validation Loss: 9.8248, Validation Accuracy: 16.96%

Epoch 2/10
Training Loss: 2.1714
Validation Loss: 2.7086, Validation Accuracy: 16.07%

Epoch 3/10
Training Loss: 2.1241
Validation Loss: 2.3975, Validation Accuracy: 18.75%

Epoch 4/10
Training Loss: 2.1219
Validation Loss: 6.0594, Validation Accuracy: 20.54%

Epoch 5/10
Training Loss: 2.0538
Validation Loss: 2.1487, Validation Accuracy: 22.32%

Epoch 6/10
Training Loss: 2.0152
Validation Loss: 1.8875, Validation Accuracy: 30.36%

Epoch 7/10
Training Loss: 2.0547
Validation Loss: 3.0728, Validation Accuracy: 24.11%

Epoch 8/10
Training Loss: 1.9527
Validation Loss: 2.0273, Validation Accuracy: 28.57%

Epoch 9/10
Training Loss: 2.0129
Validation Loss: 3.1253, Validation Accuracy: 12.50%

Epoch 10/10
Training Loss: 1.9880
Validation Loss: 3.2746, Validation Accuracy: 15.18%

